In [2]:
import pandas as pd
from astropy.io import fits
import numpy as np
from pathlib import Path
import glob

from dateutil.parser import parse

In [108]:
# MUST BE RUN IN ASTROCONDA38

from fast_combine import fast_process, marckie_slice_combine_reduce

In [4]:
%mkdir processed
%mkdir processed_uncorrected

dark_hdu = fits.open(Path('./calibs/Master_DarkBias.fits'))
#SuperDarkBias = dark_hdu[0].data
SuperDarkBias = dark_hdu[0].data.astype(np.float32)


flatC_norm_hdu = fits.open(Path('./calibs/Master_Flat_C_norm.fits'))
SuperNormFlat_C = flatC_norm_hdu[0].data.astype(np.float32)


mkdir: processed: File exists
mkdir: processed_uncorrected: File exists


In [4]:
fast_process(glob.glob('MBAFiles/*/*'), './processed/', exp_time=10, chunk_size=6,
    darkbias=SuperDarkBias, flat=SuperNormFlat_C, do_offset=True
    )

starting process
working on fileMBAFiles/20221111/Brorfelde_4_20221111_224933_1.fits
working on fileMBAFiles/20221111/Brorfelde_1_20221111_183745_0.fits
working on fileMBAFiles/20221111/Brorfelde_1_20221111_183745_1.fits
working on fileMBAFiles/20221111/Brorfelde_4_20221111_224933_0.fits
working on fileMBAFiles/20221111/Brorfelde_2_20221111_195408_0.fits
working on fileMBAFiles/20221111/Brorfelde_2_20221111_195408_1.fits
working on fileMBAFiles/20221111/Brorfelde_3_20221111_211257_1.fits
working on fileMBAFiles/20221111/Brorfelde_3_20221111_211257_0.fits
combined_2022-11-12T07:22:56.0000__Brorfelde_4_20221111_224933_1.fits
reducing image 0 from file Brorfelde_4_20221111_224933_1.fits with timestamp offset 2000
Successfully sliced and combined combined_2022-11-12T07:22:56.0000__Brorfelde_4_20221111_224933_1.fits
combined_2022-11-12T03:11:08.0000__Brorfelde_1_20221111_183745_1.fits
reducing image 0 from file Brorfelde_1_20221111_183745_1.fits with timestamp offset 2000
combined_2022-11-1

In [3]:
fast_process(glob.glob('MBAFiles/*/*'), './processed_uncorrected/', exp_time=10, chunk_size=6,
    darkbias=0, flat=1, do_offset=True
    )

starting process
working on fileMBAFiles/20221111/Brorfelde_1_20221111_183745_0.fits
working on fileMBAFiles/20221111/Brorfelde_4_20221111_224933_1.fits
working on fileMBAFiles/20221111/Brorfelde_1_20221111_183745_1.fits
working on fileMBAFiles/20221111/Brorfelde_3_20221111_211257_1.fits
working on fileMBAFiles/20221111/Brorfelde_4_20221111_224933_0.fits
working on fileMBAFiles/20221111/Brorfelde_2_20221111_195408_0.fits
working on fileMBAFiles/20221111/Brorfelde_3_20221111_211257_0.fits
working on fileMBAFiles/20221111/Brorfelde_2_20221111_195408_1.fits
combined_2022-11-12T04:27:31.0000__Brorfelde_2_20221111_195408_1.fits
reducing image 0 from file Brorfelde_2_20221111_195408_1.fits with timestamp offset 2000
combined_2022-11-12T03:11:08.0000__Brorfelde_1_20221111_183745_1.fits
reducing image 0 from file Brorfelde_1_20221111_183745_1.fits with timestamp offset 2000
combined_2022-11-12T05:46:20.0000__Brorfelde_3_20221111_211257_1.fits
reducing image 0 from file Brorfelde_3_20221111_211

In [ ]:
%mkdir combined_all

fast_process(glob.glob('MBAFiles/*/*'), './combined_all/', exp_time=10, chunk_size=6,
    darkbias=SuperDarkBias, flat=SuperNormFlat_C, do_offset=True
    )

In [3]:
# MUST BE RUN IN (astroconda3)

import alipy
from datetime import datetime



In [4]:
%mkdir aligned

def align_imgs(images_to_align, ref_image):

    identifications = alipy.ident.run(ref_image, list(images_to_align), visu=False)

    outputshape = alipy.align.shape(ref_image)

    for id in identifications: # list of the same length as images_to_align.
        if id.ok == True:
            alipy.align.affineremap(id.ukn.filepath, id.trans,
                shape=outputshape,
                makepng=True,
                outdir=f'aligned/')

            print("SUCCESS: %20s : %20s, flux ratio %.2f" % (id.ukn.name,
                                                        id.trans,
                                                        id.medfluxratio))

        else:
            print("FAILED: %20s" % (id.ukn.name))
    return identifications

mkdir: aligned: File exists


In [16]:
imgs = pd.DataFrame(
    columns=('filepath', 'timestamp', 'ref_img'),
    #dtype=(str, datetime, str)
)

def _get_time(fp:str):
    name = Path(fp).name.split('__')[0].lstrip('combined_')
    return parse(name)
    #return datetime.datetime.strptime(name, 'combined_%Y-%m-%dT%X.0000')




imgs['filepath'] = glob.glob(f"processed/*.fits")

imgs['timestamp'] = imgs['filepath'].apply(_get_time)

## Filter cell

In [18]:
# FILTER CELL

night_1 = (imgs['timestamp'] >= parse('2022-11-11T05:51:00')) & (imgs['timestamp'] < parse('2022-11-12T00:00:00'))



night_2_group_1 = (
    (parse('2022-11-12T00:00:00') <= imgs['timestamp']) & 
    (imgs['timestamp'] < parse('2022-11-12T05:59:20'))
    )

night_2_group_2 = (
    (parse('2022-11-12T06:51:36') <= imgs['timestamp']) & 
    (imgs['timestamp'] < parse('2022-11-12T07:21:36'))
    )


night_3 = (
    (parse('2022-11-13T00:00:00') <= imgs['timestamp']) & 
    (imgs['timestamp'] < parse('2022-11-14T00:00:00'))
    )


In [14]:
sum(night_2_group_2) # total number in group

75

In [25]:
imgs.loc[night_1, 'ref_img'] = './processed/combined_2022-11-11T05:51:00.0000__Brorfelde_4_20221110_215057_0.fits'

imgs.loc[night_2_group_1, 'ref_img'] = './processed/combined_2022-11-12T04:36:31.0000__Brorfelde_2_20221111_195408_1.fits'
imgs.loc[night_2_group_2, 'ref_img'] = './processed/combined_2022-11-12T06:59:36.0000__Brorfelde_4_20221111_224933_0.fits'

imgs.loc[night_3, 'ref_img'] = './processed/combined_2022-11-13T05:00:40.0000__Brorfelde_3_20221112_205136_0.fits'

imgs.dropna(inplace=True)

imgs[night_3]

/Users/student/anaconda3/envs/astroconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


,filepath,timestamp,ref_img,ok
0,processed/combined_2022-11-13T03:49:33.0000__B...,2022-11-13 03:49:33,./processed/combined_2022-11-13T05:00:40.0000_...,True
7,processed/combined_2022-11-13T03:51:33.0000__B...,2022-11-13 03:51:33,./processed/combined_2022-11-13T05:00:40.0000_...,True
11,processed/combined_2022-11-13T02:31:26.0000__B...,2022-11-13 02:31:26,./processed/combined_2022-11-13T05:00:40.0000_...,True
12,processed/combined_2022-11-13T03:35:33.0000__B...,2022-11-13 03:35:33,./processed/combined_2022-11-13T05:00:40.0000_...,True
15,processed/combined_2022-11-13T02:12:26.0000__B...,2022-11-13 02:12:26,./processed/combined_2022-11-13T05:00:40.0000_...,False
...,...,...,...,...
651,processed/combined_2022-11-13T05:08:40.0000__B...,2022-11-13 05:08:40,./processed/combined_2022-11-13T05:00:40.0000_...,False
652,processed/combined_2022-11-13T05:38:00.0000__B...,2022-11-13 05:38:00,./processed/combined_2022-11-13T05:00:40.0000_...,False
655,processed/combined_2022-11-13T04:17:53.0000__B...,2022-11-13 04:17:53,./processed/combined_2022-11-13T05:00:40.0000_...,True
657,processed/combined_2022-11-13T06:35:57.0000__B...,2022-11-13 06:35:57,./processed/combined_2022-11-13T05:00:40.0000_...,True


In [15]:
imgs.loc[night_2_group_2, 'ref_img'] = './processed/combined_2022-11-12T06:59:36.0000__Brorfelde_4_20221111_224933_0.fits'


In [26]:
from os.path import exists
# ensure all ref images are real files
imgs['ref_img'].apply(exists).all()

True

In [27]:
# perform alignment for the specified groups

for group in (night_3,):
    identifications = align_imgs(imgs[group]['filepath'], imgs[group]['ref_img'].iloc[0])
    imgs.loc[group, 'ok'] = [id.ok for id in identifications]

/Users/student/anaconda3/envs/astroconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


##########  Preparing reference ...
No filter file found, using default filter
No NNW file found, using default NNW config
Number of sources in catalog : 6
EXT_NUMBER values found in catalog : 1
I've selected 4 sources
Making more quads, from quadlevel 0 ...
Made    1 quads from    4 stars (combi n=7 s=0 d=30.0)
########## Processing processed/combined_2022-11-13T03:49:33.0000__Brorfelde_2_20221112_192530_0.fits
No filter file found, using default filter
No NNW file found, using default NNW config
Number of sources in catalog : 6
EXT_NUMBER values found in catalog : 1
I've selected 6 sources
Making more quads, from quadlevel 0 ...
Made   15 quads from    6 stars (combi n=7 s=0 d=30.0)
Removing 0/15 duplicates
Finding 4 best candidates among 15 x 1 (ukn x ref)
We have a maximum of 15 quad pairs
Cand  1 (dist.   0.00098071) : Rotation   -0.004865 [deg], scale 0.999257
Cand  2 (dist.   0.42508252) : Rotation +137.228675 [deg], scale 0.986137
Cand  3 (dist.   0.64580989) : Rotation +156.05

In [32]:
#success rate night 3
sum(imgs[night_3]['ok']) / imgs[night_3].shape[0]

/Users/student/anaconda3/envs/astroconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


0.5860655737704918

In [39]:
imgs[night_3&imgs['ok']].sort_values('timestamp').query('"2022-11-13 05:00:00" < timestamp')

/Users/student/anaconda3/envs/astroconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,filepath,timestamp,ref_img,ok
58,processed/combined_2022-11-13T05:00:40.0000__B...,2022-11-13 05:00:40,./processed/combined_2022-11-13T05:00:40.0000_...,True
500,processed/combined_2022-11-13T05:02:40.0000__B...,2022-11-13 05:02:40,./processed/combined_2022-11-13T05:00:40.0000_...,True
302,processed/combined_2022-11-13T05:03:40.0000__B...,2022-11-13 05:03:40,./processed/combined_2022-11-13T05:00:40.0000_...,True
476,processed/combined_2022-11-13T05:04:40.0000__B...,2022-11-13 05:04:40,./processed/combined_2022-11-13T05:00:40.0000_...,True
26,processed/combined_2022-11-13T05:05:40.0000__B...,2022-11-13 05:05:40,./processed/combined_2022-11-13T05:00:40.0000_...,True
288,processed/combined_2022-11-13T05:06:40.0000__B...,2022-11-13 05:06:40,./processed/combined_2022-11-13T05:00:40.0000_...,True
570,processed/combined_2022-11-13T05:07:40.0000__B...,2022-11-13 05:07:40,./processed/combined_2022-11-13T05:00:40.0000_...,True
651,processed/combined_2022-11-13T05:08:40.0000__B...,2022-11-13 05:08:40,./processed/combined_2022-11-13T05:00:40.0000_...,True
210,processed/combined_2022-11-13T05:09:40.0000__B...,2022-11-13 05:09:40,./processed/combined_2022-11-13T05:00:40.0000_...,True
398,processed/combined_2022-11-13T05:13:40.0000__B...,2022-11-13 05:13:40,./processed/combined_2022-11-13T05:00:40.0000_...,True


In [2]:
imgs.to_csv('img_master_spreadsheet.csv')

NameError: name 'imgs' is not defined

# Photometry: must be run in astroconda38

,wow,wut
0,2,4
1,6,8
